In [1]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
import time
import re
import pandas as pd

In [2]:
# add 'chromedriver' executable to PATH
import os
os.environ['PATH']+=";C:\Program Files (x86)\Google\Chrome\Application"

###  Making a POST request in Selenium using JavaScript

In [6]:
# initiate the driver
start_url="https://servidorespublicos.gob.mx/"

options = Options()
options.add_experimental_option('prefs', {
    "download.default_directory": "/home/",       # change default directory for downloads
    "download.prompt_for_download": False,        # to auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True    # it will not show PDF directly in chrome
})
#options.add_argument('--headless')

driver=webdriver.Chrome(options=options)
driver.get(start_url)
time.sleep(5)

try:
    driver.find_element_by_xpath("//*[contains(text(),'De acuerdo')]").click()  
except Exception as e:
    print(e)

In [4]:
# import the user list for search
import pandas as pd
df=pd.read_csv("Name_Info_tbd.csv",index_col=False)    # specify the file name
df.columns=['Nombre','idUsrDecnet','CUSP','RFC','href']

user_list=df[['Nombre','href','RFC']].apply(tuple,axis=1).tolist()
display(df)

,Nombre,idUsrDecnet,CUSP,RFC,href
0,VERÃNICA SOFÃA HERNÃNDEZ HERNÃNDEZ,2155594,HEHV810303MDFRRR06,HEHV8103038N2,https://servidorespublicos.gob.mx/declaracione...
1,VERÃNICA YAZMÃN HERNÃNDEZ HERNÃNDEZ,2581573,HEHV891107MMCRRR01,HEHV8911073Q0,https://servidorespublicos.gob.mx/declaracione...
2,VERONICA DEL CARMEN HERNANDEZ HERNANDEZ,2019207,HEHV810630MCSRRR03,HEHV810630QJ9,https://servidorespublicos.gob.mx/declaracione...
3,VERONICA MERIT MORENO HERNÃNDEZ MORENO HERNÃ...,2354235,MOHV810212MCSRRR02,MOHV810212699,https://servidorespublicos.gob.mx/declaracione...
4,LIDIA VERONICA HERNANDEZ,3255685,HEEL730326MDFRSD01,HEEL7303262A6,https://servidorespublicos.gob.mx/declaracione...
...,...,...,...,...,...
304081,SONIA VARGAS HERNANDEZ,2702831,VAHS750711MMCRRN05,VAHS750711LC0,https://servidorespublicos.gob.mx/declaracione...
304082,SONIA VARGAS HERNANDEZ,1378083,VAHS740414MJCRRN05,VAHS740414LC3,https://servidorespublicos.gob.mx/declaracione...
304083,SONIA VARGAS OLVERA,509665,,VAOS750918V73,https://servidorespublicos.gob.mx/declaracione...
304084,SONIA VELA SANCHEZ,2967110,VESS620419MTSLNN06,VESS620419HD3,https://servidorespublicos.gob.mx/declaracione...


In [7]:
# create a dictionary to map each individual ID ('RFC') to a document ID
dict_mapping={}

# open the servants' page on separate tabs and download pdf
wait = WebDriverWait(driver, 10)
for item in user_list:   
    
    link = item[1]
    command = "window.open('" + link + "');"
    # open a new tab with the .pdf list
    driver.execute_script(command)
    # switch to the new opened tab
    driver.switch_to.window(driver.window_handles[-1])     
    # get document ID（Número de comprobante）
    documentID = wait.until(ec.visibility_of_element_located((By.XPATH,"//*[@id='tableDeclaracion']/tbody/tr[1]/td[3]"))).text
    dict_mapping[item[2]] = documentID
    # download only pdfs from 2020 or later
    if documentID[0:4] == "2021" or documentID[0:4] == "2020":
        # click on the .pdf link button
        time.sleep(2)
        text_menu = wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "text-info")))
        text_menu.click()
        # download and save the .pdf
        time.sleep(5)
        download_btn = wait.until(ec.visibility_of_element_located((By.ID, "download")))
        download_btn.click()
    # clean up the desk and re-start
    #  close the new tab
    driver.close()     
    #  change the focus to the main page
    driver.switch_to.window(driver.window_handles[0])     

TimeoutException: Message: 


In [8]:
# save the mapping info as dataframe
mapping_table = pd.Series(dict_mapping).to_frame().reset_index()
mapping_table.columns = ['Individual ID','Document ID']
display(mapping_table)

# export the dataframe as csv file
mapping_table.to_csv('Outcomes/3. pdf_collection/Individual_Document_ID_13.csv')

,Individual ID,Document ID
0,HEHV8103038N2,20210517163328002155594720
1,HEHV8911073Q0,20210528083403002581573178
2,HEHV810630QJ9,20210514194228002019207413
3,MOHV810212699,20210512235019002354235971
4,HEEL7303262A6,20201209194825003255685641
...,...,...
1629,EOPV520719I87,201308141303301000000208191339
1630,EUPV600916MH0,201507101817308614225
1631,EAPV6809197X0,201306111922341000000208196854
1632,EUPV720721GC8,20210109165502003492144768
